# Business Case #5 - Retail - Pre_processing

## Authors:
#### Débora Santos (m20200748),Pedro Henrique Medeiros (m20200742), Rebeca Pinheiro (m20201096)

#### Group D - D4B Consulting

### Installing and import packages

Maybe it will be necessary install some 'special' packages to this notebook works. 

Please follow the next cells and check if it's necessary

In [1]:
#IMPORT LIBRARIES
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from datetime import datetime
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans, AgglomerativeClustering, MeanShift, DBSCAN, estimate_bandwidth
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder, RobustScaler
from itertools import product
from math import ceil
from scipy.cluster.hierarchy import dendrogram
%matplotlib inline
from pandas_profiling import ProfileReport 
%config InlineBackend.figure_format = 'retina' 
from scipy.stats import iqr as IQR
from collections import Counter

from sklearn.base import clone
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score
import phik
import matplotlib.cm as cm
from sklearn.manifold import TSNE

#outliers detection
from scipy import stats
from sklearn.neighbors import LocalOutlierFactor
from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.tree import DecisionTreeClassifier

from os.path import join


from sklearn.metrics import pairwise_distances

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

# Seeting seaborn style
sns.set()

Collect initial data

In [2]:
#import dataset in csv
df = pd.read_csv('NOVAIMS_MAA_2020e21_BusinessCasesDataScience_MindOverData_RetailChallenge.csv')
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

KeyboardInterrupt: 

### Describe, explore and assess data quality

In [ ]:
# First look at the dataframe
df.head()

In [ ]:
# First look at the dataframe
df.tail()

In [ ]:
# Data types and null values verification
df.info()

In [ ]:
# Summary statistics for all variables
df.describe(include='all').transpose()

In [ ]:
# Replace possible empty values with NaN
df.replace("", np.nan, inplace=True)

# Function to show missing values and their percentages
def missing_data(df):
    l = []
    for col in df.columns:
        missing = df[col].isnull().sum(axis=0)
        percentage = missing/df.shape[0]
        l.append([col,missing,percentage*100])
        df_missing = pd.DataFrame(l, columns = ['Feature','missing','percentage'])
        df_missing = df_missing.sort_values('missing',axis=0, ascending = False)    
    return df_missing

# Check the percentage of missing values
df_missing = missing_data(df)
df_missing = df_missing[df_missing['missing']>0]
df_missing

## Data Preparation:

In [ ]:
df1 = df.copy()

### Feature Engineering

In [ ]:
# Change 'Date' data type to datetime
df1['Date'] = df1['Date'].astype('datetime64[ns]')

In [ ]:
df1.info()

In [ ]:
df2 = df1[df1['Measures']== "Sell-out units"]
df2.rename(columns = {'Value':'Quantity'}, inplace = True)
df2

In [ ]:
df3 = df1[df1['Measures']== "Sell-out values"]
df3.rename(columns = {'Value':'Sales Values'}, inplace = True)
df3

In [ ]:
df2.drop(['Measures'], axis=1, inplace=True)
df3.drop(['Measures'], axis=1, inplace=True)
df2.reset_index(inplace=True)
df3.reset_index(inplace=True)

In [ ]:
df2['Sales Values'] = df3['Sales Values']
df2

In [ ]:
df2 = df2.drop(columns=['index'])
df1 = df2.copy()

In [ ]:
# Summary statistics 
df1.describe().apply(lambda s: s.apply(lambda x:format(x,'g')))

In [ ]:
df1 = df1[df1['Sales Values'] >0]

In [ ]:
df1 ['Avg price'] = df1['Sales Values']/df1['Quantity']

In [20]:
df1.to_csv(os.path.join("df_pre_processed.csv"), index=False)

In [21]:
# Define metric features
metric_features = ['Quantity','Sales Values','Avg price']

### Exporting file to predict demand forecast

In [22]:
df_demand = df1.copy()
df_demand = df_demand[['ProductPackSKU_ID','Point-of-Sale_ID','Date','Quantity']]
df_demand

,ProductPackSKU_ID,Point-of-Sale_ID,Date,Quantity
0,ProductSKU_1970,POS_1,2017-03-04,2.0
1,ProductSKU_1970,POS_1,2016-05-02,4.0
2,ProductSKU_1970,POS_1,2016-10-24,2.0
3,ProductSKU_1970,POS_1,2017-10-13,2.0
4,ProductSKU_1970,POS_1,2017-10-14,2.0
...,...,...,...,...
91171147,ProductSKU_1813,POS_410,2016-01-28,1.0
91171148,ProductSKU_1813,POS_410,2016-04-20,1.0
91171149,ProductSKU_1813,POS_410,2016-04-25,1.0
91171150,ProductSKU_1813,POS_410,2016-04-28,1.0


In [23]:
df_demand.to_csv(os.path.join("df_demand.csv"), index=False)

# DAQUI EM DIANTE NÃO ESTÁ ATUALIZADO

### Dealing with Outliers

In [ ]:
# Histograms for the metric features
sns.set()

# Prepare figure. Create individual axes where each histogram will be placed
fig, axes = plt.subplots(3, ceil(len(metric_features) / 3), figsize=(20, 11))

# Plot data
# Iterate across axes objects and associate each histogram:
for ax, feat in zip(axes.flatten(), metric_features): 
    ax.hist(df1[feat], color = "Navy")
    ax.set_title(feat)
    
# Layout
# Add a centered title to the figure:
title = "Numeric Variables Histograms"
plt.suptitle(title)
#Adjust the space between plots
plt.subplots_adjust(wspace=0.3, hspace=0.3)


#Show Plot
plt.show()

In [ ]:
# Boxplots for the metric features
sns.set()

# Prepare figure and create individual axes where each box plot will be placed
fig, axes = plt.subplots(3, ceil(len(metric_features) /3), figsize=(20, 11))

# Plot the data
# Iterate across axes objects and associate each box plot
for ax, feat in zip(axes.flatten(), metric_features):
    sns.boxplot(df1[feat], ax=ax,color="royalblue")
    
# Layout
# Add a centered title to the figure
title = "Numeric Variables - Box Plots"

plt.suptitle(title)

# Adjust the space between plots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Show plots
plt.show()

In [ ]:
df1[df1['Avg price']>20000].sort_values(by = 'Avg price')

In [ ]:
df1[df1['Avg price']>20000].sort_values(by = 'Avg price')['ProductPackSKU_ID'].unique()

In [ ]:
df1.groupby(['ProductPackSKU_ID']).mean()['Avg price'].sort_values().tail(20)

The SKU's 4295,8223, 3242, 4212, 4206 and 3241 don't have average price around 20.000 but they had sales with price above 20.000. Let's check it.

In [ ]:
df1[df1['ProductPackSKU_ID']=='ProductSKU_4295'].sort_values(by='Avg price')

In [ ]:
df1[df1['ProductPackSKU_ID']=='ProductSKU_8223'].sort_values(by='Avg price')

In [ ]:
df1[df1['ProductPackSKU_ID']=='ProductSKU_3242'].sort_values(by='Avg price')

In [ ]:
df1[df1['ProductPackSKU_ID']=='ProductSKU_4212'].sort_values(by='Avg price')

In [ ]:
df1[df1['ProductPackSKU_ID']=='ProductSKU_4206'].sort_values(by='Avg price')

In [ ]:
df1[df1['ProductPackSKU_ID']=='ProductSKU_3241'].sort_values(by='Avg price')

The SKUs 4295, 8223, 3241 and 3242 have transactions with prices very different from the others. We are going to remove them

In [ ]:
df1 = df1[df1['Avg price']<40000]

In [ ]:
df1 = df1[~((df1['ProductPackSKU_ID']=='ProductSKU_3241')&(df1['Avg price']>20000))]

In [ ]:
df1 = df1[~((df1['ProductPackSKU_ID']=='ProductSKU_3242')&(df1['Avg price']>20000))]

In [ ]:
df1 = df1[~((df1['ProductPackSKU_ID']=='ProductSKU_4295')&(df1['Avg price']>30000))]

In [ ]:
df1

In [ ]:
# Boxplots for the metric features
sns.set()

# Prepare figure and create individual axes where each box plot will be placed
fig, axes = plt.subplots(3, ceil(len(metric_features) /3), figsize=(20, 11))

# Plot the data
# Iterate across axes objects and associate each box plot
for ax, feat in zip(axes.flatten(), metric_features):
    sns.boxplot(df1[feat], ax=ax,color="royalblue")
    
# Layout
# Add a centered title to the figure
title = "Numeric Variables - Box Plots"

plt.suptitle(title)

# Adjust the space between plots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Remove the last  2 plots
#axes.flatten()[-2].remove()
#axes.flatten()[-1].remove()
# Show plots
plt.show()

In [ ]:
df1[df1['Quantity']>150].sort_values(by = 'Quantity')

In [ ]:
df1[df1['ProductPackSKU_ID']=='ProductSKU_1572'].sort_values(by='Avg price')

In [ ]:
#Standardize the data to run the outliers methods
#Select data
df1_out = df1[metric_features].copy()
# Create a StandardScaler
scale = StandardScaler()
# Fit and Transform data by applying the scale obtained in the previous command
scale_feat= scale.fit_transform(df1_out[metric_features])
#Applying the transformation in the dataset
df1_out[metric_features]=scale_feat
#Check the results
df1_out.head()

In [ ]:
df1_out = df1[metric_features].copy()

In [ ]:
#Check the statistics summary to check it worked
df1_out.describe().apply(lambda s: s.apply(lambda x:format(x,'g')))

#### We use 6 methods to recognize outliers:

In [ ]:
#1) Zscore that checks how many standard deviations is a datapoint distant from the mean;
def out_zscore(df, threshold = 3.0):
    z = pd.DataFrame(np.abs(stats.zscore(df)), columns = df.columns, index = df.index)
    z = pd.DataFrame(z > threshold)
    return z.any(axis = 1).astype(int)

In [ ]:
#2) InterQuantile Range method that creates boundaries using the first and thirs quantile and the interquantile range;
def out_iqr(df, multiplier = 1.5):
    q75 = df.quantile(0.75, axis = 0)
    q25 = df.quantile(0.25, axis = 0)
    iqr = q75 - q25
    lower_bound = q25 - multiplier*iqr
    upper_bound = q75 + multiplier*iqr
    return df.apply(lambda x: np.any((x<lower_bound) | (x>upper_bound)), 1).astype(int)

#### 3) Local Outlier Factor (LOF) is a function that detects outliers by comparing the density of the neighborhood of a point to the ones of its neighbors.

In [ ]:
lof = LocalOutlierFactor(contamination = 'auto',metric = 'euclidean',n_neighbors=1)
lof_out = pd.Series(lof.fit_predict(df1_out), index = df1_out.index)
lof_out.value_counts(normalize = True)

#### 4) At the basis of the Isolation Forest algorithm there is the tendency of anomalous instances in a dataset to be easier to separate from the rest of the sample (isolate), compared to normal points. In order to isolate a data point the algorithm recursively generates partitions on the sample by randomly selecting an attribute and then randomly selecting a split value for the attribute, between the minimum and maximum values allowed for that attribute. When all the trees are grown outliers are identified as points easier to isolate, therefore with a smaller path lenght in the tree, being closer to the root.

In [ ]:
isolation_forest = IsolationForest(random_state = 7, contamination=  "auto")
isolation_forest.fit(df1_out) #same results with scaled
isofor_outliers = pd.Series(isolation_forest.predict(df1_out), index = df1_out.index)
isofor_outliers.value_counts(normalize = True)

#### 5) One-Class SVM is similar to support vector machine, but instead of using a hyperplane to separate two classes of instances, it uses a hypersphere to encompass all of the instances. The algorithm will try to find the smallest possible hypersphere and point outside of it will be considered outliers. One bad thing of this algorithm is that you have to set in advance the percentage of points that you think are outliers, in this case I use the one retrieved from LOF.

In [ ]:
outliers_fraction =   0.063
auto_detection = svm.OneClassSVM(kernel='rbf', gamma=0.01, degree=3, nu=outliers_fraction)
auto_detection.fit(df1_out)
svm_outliers = pd.Series(auto_detection.predict(df1_out), index = df1_out.index)
svm_outliers.value_counts(normalize = True)

#### 6) DBSCAN set a point p and the density around a point can be measured by the number of points surrounding it. To measure the density around a point p, it used the topological definition of neighborhood. The 𝜀-neighborhood of a point p is the space within a radius 𝜀 > 0 centered in p. For this algorithm, it's necessary set the minimal number of points in each neighborhood (min_samples) and the size of the radius (area) of neighborhood (eps). It's considered a core point when a point there is at least the minimal points around it into your radius. Not core points are points inside the radium of a core point, but they don't have the minimal number of points around them. The noise point are points they are neither a core point and not core point. We are interest in this analyse to detect the noise points. The 2 plots bellow are to find the eps to run the dbscan algorithm.  

In [ ]:
#Elbow plot to find the number of neighbors
inert = []
K = range(1,5)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(df1_out)
    inert.append(kmeanModel.inertia_)
#Define name for x axis and y axis
plt.xlabel("clusters")
plt.ylabel("avg_square distance")
plt.plot(K,inert,"o-",color="blue")

In [ ]:
# K-distance graph to find out the right eps value
#Use the number of neighbors detect in the elbow plot. 
neigh = NearestNeighbors(n_neighbors=3)
neigh.fit(df1_out)
distances, _ = neigh.kneighbors(df1_out)
distances = np.sort(distances[:, -1])
plt.plot(distances)
# plt.subplot(1.1,figsize=(20,15))
plt.figure(1, figsize=(20, 10))
plt.show()

In [ ]:
#After find the eps value, we run the DBSCAN algorithm
dbscan = DBSCAN(eps=2.5, min_samples=20, n_jobs=-1)
dbscan_labels = dbscan.fit_predict(df1_out)
dbscan_outliers = pd.Series(dbscan.fit_predict(df1_out), index = df1_out.index)
dbscan_outliers.value_counts(normalize = True)

In [ ]:
#Including columns with the results of each method in the dataset to compare the different methods and also combine them.
df1_out['zscore'] = out_zscore(df1_out)

df1_out['iqr'] = out_iqr(df1_out, multiplier = 3.0)

df1_out['lof_out'] = [0 for i in range(len(df1_out))]
df1_out.loc[lof_out[lof_out == -1].index, 'lof_out'] = 1

df1_out['isofor_out'] = [0 for i in range(len(df1_out))]
df1_out.loc[isofor_outliers[isofor_outliers == -1].index, 'isofor_out'] = 1

df1_out['svm_out'] = [0 for i in range(len(df1_out))]
df1_out.loc[svm_outliers[svm_outliers == -1].index, 'svm_out'] = 1

df1_out['dbscan_out'] = [0 for i in range(len(df1_out))]
df1_out.loc[dbscan_outliers[dbscan_outliers != -1].index, 'dbscan_out'] = 0
df1_out.loc[dbscan_outliers[dbscan_outliers == -1].index, 'dbscan_out'] = 1

df1_out['sum_out']  = df1_out[['zscore','iqr','svm_out','lof_out','isofor_out','dbscan_out']].apply(lambda x: sum(x), axis = 1)

In [ ]:
#Create a dataset for see the outliers detect by each model
outliers = df1_out[['zscore','iqr','svm_out','lof_out','isofor_out','dbscan_out']]
outliers.groupby(['zscore','iqr','svm_out','lof_out','isofor_out','dbscan_out'])\
            .size()\
            .to_frame()

In [ ]:
df1_out['iqr'] = out_iqr(df1_out, multiplier = 1.5)

In [ ]:
#Show the number of outliers detect by z-score individually
df1_out['zscore'].value_counts()

In [ ]:
#Show the number of outliers detect by IQR individually
df1_out['iqr'].value_counts()

In [ ]:
#Show the number of outliers detect by combined methods. 
df1_out['sum_out'].value_counts()

In [ ]:
#Chose the number 4 with the best possible solution of combined methods
df1_out2 = df1.loc[df1_out.query('sum_out == 4').index]
df2 = df1.drop(index = df1_out.query('sum_out == 4').index)

In [ ]:
#Look at the results
df2

In [ ]:
#Check boxplots after outliers removal
# Box plots for the metric features
sns.set()

# Prepare figure and create individual axes where each box plot will be placed
fig, axes = plt.subplots(4, ceil(len(metric_features) /4), figsize=(20, 11))

# Plot the data
# Iterate across axes objects and associate each box plot
for ax, feat in zip(axes.flatten(), metric_features):
    sns.boxplot(df2[feat], ax=ax,color="royalblue")
    
# Layout
# Add a centered title to the figure
title = "Numeric Variables - Box Plots"

plt.suptitle(title)

# Adjust the space between plots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Remove the last  2 plots
axes.flatten()[-2].remove()
axes.flatten()[-1].remove()
# Show plots
plt.show()

#### After testing removing based on 4 combined methods and realising that did not seem to make a big difference on the box plots in terms of outliers, we have dropped this approach and decided to apply Inter quartile range (IQR) only on five features 

#### For the features Drywh, Freq the best multiplier is 1 and for the features Dessert,Sweetred,Sweetwh, the best multiplier is 4.5

In [ ]:
#Create a copy of the dataset
df1_iqr = df1.copy()
#Define the features to apply the IQR method
iqr_features= ['Drywh','Freq']
#Run the function setting the multiplier =1 and create a new columns to store the results
df1_iqr['out'] = out_iqr(df1_iqr[iqr_features], multiplier = 1)
#Show the results
df1_iqr['out'].value_counts()

In [ ]:
#Filter the data excluding the outliers detected
df1_iqr1 = df1_iqr[df1_iqr['out']==0]
#Create a dataset to store the outliers detected
df1_iqr_out = df1_iqr[df1_iqr['out']==1]
#Drop the column create in both datasets
df1_iqr1.drop(['out'], axis=1, inplace=True)
df1_iqr_out.drop(['out'], axis=1, inplace=True)

In [ ]:
#Second step
#Define the features to apply the IQR method
iqr_features2= ['Dessert','Sweetred','Sweetwh']
#Run the function setting the multiplier =4.5 and create a new columns to store the results
df1_iqr1['out'] = out_iqr(df1_iqr1[iqr_features2], multiplier = 4.5)
#Show the results
df1_iqr1['out'].value_counts()

In [ ]:
#Filter the data excluding the outliers detected
df1_iqr2 = df1_iqr1[df1_iqr1['out']==0]
#Create a dataset to the outliers detected
df1_iqr_out2 = df1_iqr1[df1_iqr1['out']==1]
#Drop the column create in both datasets
df1_iqr2.drop(['out'], axis=1, inplace=True)
df1_iqr_out2.drop(['out'], axis=1, inplace=True)

In [ ]:
print('Percentage of data kept after removing outliers:', np.round(df1_iqr2.shape[0] / df1.shape[0], 4))

In [ ]:
#Check the results
df1_iqr2.info()

In [ ]:
#Merging the both datasets used to store the outliers detected
df1_out_final = df1_iqr_out.append(df1_iqr_out2)

In [ ]:
#Check the outliers dataset
df1_out_final.info()

In [ ]:
#Check boxplots after outliers removal
# Box plots for the metric features
sns.set()

# Prepare figure and create individual axes where each box plot will be placed
fig, axes = plt.subplots(4, ceil(len(metric_features) /4), figsize=(20, 11))

# Plot the data
# Iterate across axes objects and associate each box plot
for ax, feat in zip(axes.flatten(), metric_features):
    sns.boxplot(df1_iqr2[feat], ax=ax,color="royalblue")
    
# Layout
# Add a centered title to the figure
title = "Numeric Variables - Box Plots"

plt.suptitle(title)

# Adjust the space between plots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Remove the last  2 plots
axes.flatten()[-2].remove()
axes.flatten()[-1].remove()
# Show plots
plt.show()

In [ ]:
#Replace df1 by the method we decided
#IQR for the  5 variables. 
df1 = df1_iqr2.copy()

In [ ]:
#Check the data
df1.info()